## Using ICESat-2 ATL15, Gridded Arctic Land Ice Height to investigate ice-surface height anomalies

#### Written by Wilson Sauthoff (https://wsauthoff.github.io)

#### Key learning outcomes:
- How to gather data from disparate sources
- What is a Coordinate Reference System (CRS) and why it matters.
- How to use geometries including Points and Polygons to define an area of interest and subset data. 
- The basics of how the icepyx library and simplify interacting with ICESat-2 data. 
- How Xarray can simplify the import of multi-dimensional data.
- Open, plot, and explore gridded raster data.

#### Practical skills covered to reach key learning outcomes:
- Practice importing non-cloud-hosted, cloud-hosted, and available-via-url data to the CryoCloud Jupyter hub
- Find CRS information in datasets and reproject another CRS to plot with a dataset with a different CRS.
- Create Shapely geometries including Points and Polygons and store in a GeoPandas GeoDataFrames with other data. 
- Use the icepyx library to simplify the search for ICESat-2 data and the authenication process to access AWS s3 cloud-hosted data.
- Import multi-dimensional data using Xarray, access metadata, and use built-in Xarray methods for analysis and data visualization.
- Learn the basics of NASA's Ice Cloud, and land Elevation Satellite 2 (ICESat-2) laser altimetry mission.
- Learn the details of ICESat-2's high-level data product, ATL15 Gridded Antarctic and Arctic Land Ice Height Change.

## Computing environment

We'll be using the following open-source Python libraries in this notebook:

In [ ]:
# cells to run command line commands to work off of the icepyx development branch to get the latest features

In [ ]:
# !git clone https://github.com/icesat2py/icepyx.git

In [ ]:
# !cd path/to/repo

In [ ]:
# !pip install .

In [ ]:
# import internal libraries
import os

# import external libraries
import geopandas as gpd
import h5py
import hvplot.xarray
import icepyx as ipx
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
# %matplotlib widgets  # for interactive plots
# %matplotlib notebook  # for static plots
import numpy as np
import pandas as pd
from pyproj import CRS, Transformer
import rioxarray
import s3fs
import xarray as xr

# define utility function
def ll2ps(lon, lat):
    """
    Transform coordinates from geodetic coordinates (lon, lat)
    to Greenland (epsg:3413) coordinates (x, y)
    x, y = ll2ps(lon, lat)
    Inputs
    * lon, lat in decimal degrees (lon: W is negative; lat: S is negative)
    Outputs
    * x, y in [m]
    """
    crs_ll = CRS("EPSG:4326")
    crs_xy = CRS("EPSG:3413")
    ll_to_xy = Transformer.from_crs(crs_ll, crs_xy, always_xy = True)
    x, y = ll_to_xy.transform(lon, lat)
    return x, y

## In this tutorial, we will focus on Greenland, where active subglacial lakes have been inferred from surface height changes

Northern hemisphere subglacial lakes compiled in a global inventory by Livingstone and others (2020):

<img src="Livingstone_2020_Fig3a.png" alt="2022 inventory of subglacial lakes" class="bg-primary mb-1" width="600px">

We can investigate active subglacial lakes, which drain and fill episodically, using ice surface height anomalies as the overlying ice deforms when active lakes drain and fill. [These videos](https://svs.gsfc.nasa.gov/4913) from NASA's Science Visualization Studio illustrate how we observe active lakes filling and draining through time. 

## Locating the Greenland active subglacial lakes
Consulting the supplementary data of the [Livingstone and others (2020) inventory](https://www.nature.com/articles/s43017-021-00246-9#Sec16), we can construct a [geopandas geodataframe](https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html) to investigate Greenland's subglacial lakes.

If we are working with a dataset that is not cloud hosted or unavailable via a URL, we can upload that dataset to CryoCloud. 

First, decide where the uploaded data will live on your CryoCloud hub. It could be a data directory folder that you create in your CryoCloud's base directory or put the data into your working directory (whichever file management technique you prefer). 

Second, download the paper's supplementary data by [clicking here](https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx). 

Third, upload the supplementary data spreadsheet into your new data directory folder or your working directory. The supplementary data spreadsheet (.xlsx) is already uploaded to our working directory. 

In [ ]:
# Read in spreadsheet using pandas read_excel
use_cols = ['Name / Location', 'Lat. oN', 'Lon. oE', 'Lake Type', 'References']
import_rows = np.arange(0,65)
# You may add a path/to/file if you'd like to try reading spreadsheet from where you saved it in a data directory 
df = pd.read_excel('43017_2021_246_MOESM1_ESM.xlsx', 'Greenland', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View pandas dataset head (first 5 rows)
df.head()

However, since this data is a direct download URL, we can read the data directly into CryoCloud without the tedious download and upload steps, like so: 

In [ ]:
# Read in spreadsheet using pandas read_excel
url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx'
use_cols = ['Name / Location', 'Lat. oN', 'Lon. oE', 'Lake Type', 'References']
import_rows = np.arange(0,65)
df = pd.read_excel(url, sheet_name='Greenland', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View pandas dataset head (first 5 rows)
df.head()

This is looking good, but we can make it even better by storing the data in a GeoPandas GeoDataFrame which offers additional functionality beyond pandas of a geometry column of Shapely objects, here Shapely points: 

In [ ]:
# Create GeoPandas GeoDataFrame from Pandas DataFrame
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['Lon. oE'], df['Lat. oN']))

# Set the Coordinate Reference System (CRS) of the geodataframe
if gdf.crs is None: 
    # set CRS WGS84 in lon, lat
    gdf.set_crs('epsg:4326', inplace=True)
    
# Display GeoDataFrame
gdf

In [ ]:
# Let's look Greenland's active subglacial lake inventory
gdf[gdf['Lake Type'] == 'Active']

### What is a CRS/EPSG?

- CRS or epsg are the projection of the data onto a map.
- There are numerous formats that are used to document a CRS. Three common formats include: proj.4, EPSG, and Well-known Text (WKT) formats.
- Often you have CRS information in one format and you need to transform.
- One of the most powerful websites to look up CRS strings is Spatialreference.org. You can use the search on the site to find an EPSG code.
- A particular CRS can be referenced by its EPSG code (i.e., epsg:4121). The EPSG is a structured dataset of CRS and Coordinate Transformations. It was originally compiled by the now defunct European Petroleum Survey Group (EPSG).

In [ ]:
# Load "Natural Earth” countries dataset, bundled with GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
Greenland = world[world['name'] == 'Greenland']
Greenland = Greenland.to_crs('epsg:3413')
    
# Create a rough plot to ensure data was read properly
fig, ax = plt.subplots(figsize=(5,5))
Greenland.plot(ax=ax, facecolor='lightgray', edgecolor='gray')
gdf[gdf['Lake Type'] == 'Active'].to_crs('epsg:3413').plot(ax=ax, label='active subglacial lake')
ax.set_xlabel('northing (m)'); ax.set_ylabel('easting (m)')
ax.set_title('Active subglacial lake distribution \nacross Greenland')
plt.legend()
plt.show()

### Why we want geometries?


In [ ]:
# Create GeoSeries of our GeoDataFrame that converts to Arcic polar stereographic projection 
# and makes 10-km radius buffered polygon around each lake point
gs = gdf.to_crs('3995').buffer(10000)

# Create new GeoDataFrame to store the polygons
gdf_polys = gpd.GeoDataFrame(gdf, geometry=gs, crs='epsg:3995').to_crs('4623')

# Look at active lakes to ensure it worked as expected
gdf_polys[gdf_polys['Lake Type'] == 'Active']

## What is ICESat-2?

ICESat-2 (Ice, Cloud, and land Elevation Satellite 2), part of NASA's Earth Observing System, is a satellite mission for measuring ice sheet elevation and sea ice thickness, as well as land topography, vegetation characteristics, and clouds. It does so using an altimeter or an altitude meter, which is an instrument used to measure the altitude of an object above a fixed level. This is typically achieved by measuring the time it takes for a lidar or radar pulse, released by a satellite-based altimeter, to travel to a surface, reflect, and return to be recorded by an onboard instrument. ICESat-2 uses three pairs of laser pulsers and the detector to count the reflected photons. 

ICESat-2 laser configuration (from Smith and others, 2019). 

<img src="Smith_2019_fig1.jpg" alt="ICESat-2 laser configuration" class="bg-primary mb-1" width="1000px">


## What is ATL15?
ATL15 is one of the various [ICESat-2 data products](https://icesat-2.gsfc.nasa.gov/science/data-products). ATL15 provides various resolutions (1 km, 10 km, 20 km, and 40 km) height-change maps at 3-month intervals, allowing for visualization of height-change patterns and calculation of integrated regional volume change or integrated active subglacial lake volume change (Smith and others, 2022). 

## Streaming cloud-hosted data from NASA Earth Data Cloud
We will be working with cloud-hosted data files. This [guide](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide) explains how to find and access Earthdata cloud-hosted data. And [here](https://nsidc.org/data/earthdata-cloud) is a complete list of earthdata cloud-hosted data products available from NSIDC.

In [ ]:
# export to geopackage to subset ICESat-2 data
gdf_polys[gdf_polys['Name / Location'] == 'Flade Isblink ice cap'].to_file(os.getcwd() + '/Flade_Isblink_poly.gpkg')

## Using icepyx to simplify searching for ICESat-2 data and authenicating with AWS s3 login credentials

In [ ]:
# Specifying the necessary icepyx parameters
short_name = 'ATL15'
spatial_extent = 'Flade_Isblink_poly.gpkg' 
date_range = ['2018-09-15','2023-03-02']

In [ ]:
# Setup the Query object
region = ipx.Query(short_name, spatial_extent, date_range)

In [ ]:
# Visualize area of interest
region.visualize_spatial_extent()

In [ ]:
# Let's find the available data granuales (files)
gran_ids = region.avail_granules()
gran_ids

In [ ]:
# Let's see the granuales IDs and if they are available on the cloud
gran_ids = region.avail_granules(ids=True, cloud=True)
gran_ids

In [ ]:
# s3url = gran_ids[1][0]
# s3url

icepyx doesn't have the s3 urls for the ATL15 data product yet, so we pulled these from NASA Earth Data: https://www.earthdata.nasa.gov/

Learn more about finding cloud-hosted data from NASA Earth data cloud [here](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide)

In [ ]:
s3url_ATL15 = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL15/002/2019/ATL15_GL_0314_01km_002_01.nc'
# learn more about the ICESat-2 ATL15 Gridded Antarctic and Arctic Land Ice Height Change data product dataset here: 
# https://doi.org/10.5067/ATLAS/ATL15.002

The next step requires a NASA Earth Data user account. You can register for a free account [here](https://www.earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/earthdata-login). 

In [ ]:
earthdata_uid = 'wsauthoff'  # Replace with your NASA Earth Data user ID
earthdata_email = 'wilson.sauthoff@gmail.com' # Replace with your NASA Earth Data email

# Authenicate using your NASA Earth Data login credentials; enter your password when prompted
region.earthdata_login(earthdata_uid, earthdata_email, s3token=True)

In [ ]:
credentials = region._s3login_credentials

s3 = s3fs.S3FileSystem(key=credentials['accessKeyId'],
                       secret=credentials['secretAccessKey'],
                       token=credentials['sessionToken'])

In [ ]:
# Open s3url data file and store in Xarray Dataset
# This cell takes 10s of secs to load
with s3.open(s3url_ATL15,'rb') as f:
    # ATL15_dh = rioxarray.open_rasterio(f, group='delta_h').load()  # FIXME: preferred, but giving error
    ATL15_dh = xr.open_dataset(f, group='delta_h').load()

# View Xarray Dataset
ATL15_dh

We can acquaint ourselves with this dataset in a few ways: 
- The data product's [overview page](https://doi.org/10.5067/ATLAS/ATL15.002) (Smith and others, 2022) to get the very basics such as geographic coverage, CRS, and what the data product tells us (quarterly height changes).
- The Xarray Dataset read-in metadata: clicking on the written document icon of each data variable will expand metadata including a data variable's dimensions, datatype, etc. 
- The data product's [data dictionary](https://nsidc.org/data/documentation/atl15-data-dictionary-v01) (Smith and others, 2021) to do a deep dive on what individual variables tell us. 

We'll be plotting the delta_h data variable in this tutorial, here's what we can learn about from our these sources:
- Overview page: this is likely the 'quarterly height changes' described, but let's dive deeper to be sure
- Xarray Dataset imbedded metadata tells us a couple things: height change  at 1 km (the resolution selected earlier) and height change relative to the datum (Jan 1, 2020) surface

Ok, since the data is relative to a datum, we have two options: 
1) Difference individual time slices to subtract out the datum, like so: 

    (time_0 - datum) - (time_1 - datum) = time_0 - datum - time_1 + datum = time_0 - time_1

2) Subtract out the datum directly. The datum is the complementary dataset ATL14: a high-resolution (100 m) digital elevation model (DEM) that provides spatially continuous gridded data of ice sheet surface height. [NEED TO VERIFY THIS; DATA DICTIONARY IS NOT EXPLICIT]

In this tutorial we'll use the first method. We'll use some explanatory data analysis to illustrate this.  

In [ ]:
# Let's make a simple plot of the first minus the zeroth time slices
fig, ax = plt.subplots()
dhdt = ATL15_dh['delta_h'][1,:,:] - ATL15_dh['delta_h'][0,:,:]
cb = ax.imshow(dhdt, origin='lower', norm=mpl.colors.CenteredNorm(), cmap='coolwarm_r', 
               extent=[Greenland.bounds.minx.values[0], Greenland.bounds.maxx.values[0], Greenland.bounds.miny.values[0], Greenland.bounds.maxy.values[0]])
ax.set_xlabel('northing (m)'); ax.set_ylabel('easting (m)')
plt.colorbar(cb, label='height change [m]')
plt.show()

Hmmm...doesn't look like much change over this quarter. Why? Check out the bounds of the colorbar, we've got some pretty extreme values (colorbar is defaulting to ±10 m!) that appear to be along the margin. It's making more sense now. We can change the bounds of the colorbar to plot see more of the smaller scale change in the continental interior.

In [ ]:
# Let's calculate some basic stats to determine appropriate coloarbar bounds
print(dhdt.min().values)
print(dhdt.max().values)

We can use a TwoSlopeNorm to achieve different mapping on either side the center at zero: 

In [ ]:
# We can make that same plot with more representative colorbar bounds
fig, ax = plt.subplots()
dhdt = ATL15_dh['delta_h'][1,:,:] - ATL15_dh['delta_h'][0,:,:]
divnorm = colors.TwoSlopeNorm(vmin=dhdt.min(), vcenter=0, vmax=dhdt.max())
cb = ax.imshow(dhdt, origin='lower', norm=divnorm, cmap='coolwarm_r', 
               extent=[Greenland.bounds.minx.values[0], Greenland.bounds.maxx.values[0], Greenland.bounds.miny.values[0], Greenland.bounds.maxy.values[0]])
ax.set_xlabel('northing (m)'); ax.set_ylabel('easting (m)')
plt.colorbar(cb, label='height change [m]')
plt.show()

Now the colorbar bounds are more representative, but we still have the issue that extreme values along the margin are swapping any signals we might see in the continental interior. 

In [ ]:
# Let's use the Xarray.DataArray quantile method to find the 1% and 99% quantiles (Q1 and Q3) of the data
dhdt.quantile([0.01,0.99])

We can use these quantiles as the colorbar bounds so that we see the data variability by plotting the most extreme values at the maxed out value of the colorbar. We'll adjust the caps of the colorbar to express that there are data values beyond the bounds. 

In [ ]:
# Let's make the same plot but using the quantiles as the colorbar bounds
fig, ax = plt.subplots()
dhdt = ATL15_dh['delta_h'][1,:,:] - ATL15_dh['delta_h'][0,:,:]
divnorm = colors.TwoSlopeNorm(vmin=dhdt.quantile(0.01), vcenter=0, vmax=dhdt.quantile(0.99))
cb = ax.imshow(dhdt, origin='lower', norm=divnorm, cmap='coolwarm_r', 
               extent=[Greenland.bounds.minx.values[0], Greenland.bounds.maxx.values[0], Greenland.bounds.miny.values[0], Greenland.bounds.maxy.values[0]])
ax.set_xlabel('northing (m)'); ax.set_ylabel('easting (m)')
plt.colorbar(cb, extend='both', label='height change [m]')
plt.show()

In [ ]:
# Let's make the same plot but for all the available time slices and let's turn it in a function so that we can reuse this code for a smaller subset of data
# create empty lists to store data
def plot_icesat2_atl15(xmin, xmax, ymin, ymax, dataset):
    # subset data using bounding box in epsg:3134 x,y
    mask_x = (dataset.x >= xmin) & (dataset.x <= xmax)
    mask_y = (dataset.y >= ymin) & (dataset.y <= ymax)
    ds_sub = dataset.where(mask_x & mask_y, drop=True)
    
    # Create empty lists to store data
    vmins_maxs = []

    # Find the min's and max's of each inter time slice comparison and store into lists
    for idx in range(len(ds_sub['time'].values)-1): 
        dhdt = ds_sub['delta_h'][idx+1,:,:] - ds_sub['delta_h'][idx,:,:]
        vmin=dhdt.quantile(0.01)
        vmins_maxs += [vmin]
        vmax=dhdt.quantile(0.99)
        vmins_maxs += [vmax]
        if (min(vmins_maxs)<0) & (max(vmins_maxs)>0):
            vcenter = 0
        else: 
            vcenter = max(vmins_maxs) - min(vmins_maxs)
        divnorm = colors.TwoSlopeNorm(vmin=min(vmins_maxs), vcenter=vcenter, vmax=max(vmins_maxs))

    # create fig, ax
    fig, axs = plt.subplots(2,7, sharex=True, sharey=True, figsize=(10,4))

    idx = 0
    for ax in axs.ravel():   
        ax.set_aspect('equal')
        dhdt = ds_sub['delta_h'][idx+1,:,:] - ds_sub['delta_h'][idx,:,:]
        cb = ax.imshow(dhdt, origin='lower', norm=divnorm, cmap='coolwarm_r', 
                       extent=[Greenland.bounds.minx.values[0], Greenland.bounds.maxx.values[0], Greenland.bounds.miny.values[0], Greenland.bounds.maxy.values[0]])
        # Change polar stereographic m to km
        km_scale = 1e3
        ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
        ax.xaxis.set_major_formatter(ticks_x)
        ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
        ax.yaxis.set_major_formatter(ticks_y)
        # Create common axes labels
        fig.supxlabel('easting (m)'); fig.supylabel('northing (m)')
        # Increment the idx
        idx = idx + 1
     
    fig.colorbar(cb, ax=axs.ravel().tolist())
    plt.show()

In [ ]:
# Let's zoom into an individual active lake to see more detail
gdf_polys[gdf_polys['Lake Type'] == 'Active']

In [ ]:
# We can call the bounds of the geometry of the Shapely Polygon we created earlier
gdf_sub = gdf_polys[gdf_polys['Name / Location'] == 'Inuppaat Quuat']
gdf_sub.geometry.bounds

In [ ]:
lon_min = gdf_sub.geometry.bounds.minx
lon_max = gdf_sub.geometry.bounds.maxx
lat_min = gdf_sub.geometry.bounds.miny
lat_max = gdf_sub.geometry.bounds.maxy
xmin, ymin = ll2ps(lon_min, lat_min)
xmax, ymax = ll2ps(lon_max, lat_max)

plot_icesat2_atl15(xmin, xmax, ymin, ymax, ATL15_dh)

In [ ]:
# Let's use Xarray to export the time series into a gif
# PLACEHOLDER

In [ ]:
# Let's explore plotting the data interactively using Xarray and Holoviews
hvplot.extension('matplotlib')
divnorm = colors.TwoSlopeNorm(vmin=-5, vcenter=0, vmax=5)
ATL15_dh['delta_h'].hvplot(groupby='time', cmap='coolwarm_r', norm=divnorm, invert=True, 
                           width=(ATL15_dh['x'].max()-ATL15_dh['x'].min())/3e3, height=(ATL15_dh['y'].max()-ATL15_dh['y'].min())/3e3, 
                           widget_type='scrubber', widget_location='bottom')

In [ ]:
# clean up environment by deleting intermediary files
os.remove('Flade_Isblink_poly.gpkg')

## Streaming cloud-hosted data via EarthAccess (PLACEHOLDER)

## References

Livingstone, S.J., Li, Y., Rutishauser, A. et al. Subglacial lakes and their changing role in a warming climate. Nat Rev Earth Environ 3, 106–124 (2022). https://doi.org/10.1038/s43017-021-00246-9

Smith, B., Fricker, H. A., Holschuh, N., Gardner, A. S., Adusumilli, S., Brunt, K. M., et al. (2019). Land ice height-retrieval algorithm for NASA’s ICESat-2 photon-counting laser altimeter. Remote Sensing of Environment, 233, 111352. https://doi.org/10.1016/j.rse.2019.111352

Smith, B., T. Sutterley, S. Dickinson, B. P. Jelley, D. Felikson, T. A. Neumann, H. A. Fricker, A. Gardner, L. Padman, T. Markus, N. Kurtz, S. Bhardwaj, D. Hancock, and J. Lee. (2022). ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2 [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/ATLAS/ATL15.002. Date Accessed 2023-03-16.

Smith, B., T. Sutterley, S. Dickinson, B. P. Jelley, D. Felikson, T. A. Neumann, H. A. Fricker, A. Gardner, L. Padman, T. Markus, N. Kurtz, S. Bhardwaj, D. Hancock, and J. Lee. “ATL15 Data Dictionary (V01).” National Snow and Ice Data Center (NSIDC), 2021-11-29. https://nsidc.org/data/documentation/atl15-data-dictionary-v01. Date Accessed 2023-03-16.